In [ ]:
#########################################################
#
#   1) Ingest and clean data from two sheets (basal and bolus)
#      in the Excel file "NAMED-FILE.xlsx" Obviously, rename this file below!
#   2) Chunk basal data into flexible intervals (each <=6 minutes),
#      computing the delivered insulin "Amount" (rounded to 3 decimals)
#      and carrying the original Rate as "Temp rate"
#   3) Merge bolus data (left as-is) with the chunked basal data and export 
#      the merged DataFrame to a CSV file.
#
############################################################

import pandas as pd
import numpy as np
from math import ceil
from datetime import timedelta

##############################
# 1) Ingest & Clean the Data
##############################

# Specify the sheet names for basal and bolus data:
basal_sheet_name = "Basal"   # replace with your actual basal sheet name
bolus_sheet_name = "Bolus"   # replace with your actual bolus sheet name

# Read basal data from the specified sheet
basal_df = pd.read_excel("NAMED-FILE.xlsx", 
                         sheet_name=basal_sheet_name)

# Read bolus data from the specified sheet
bolus_df = pd.read_excel("NAMED-FILE.xlsx", 
                         sheet_name=bolus_sheet_name)

# For basal, keep these columns:
# Local Time, Tidepool Data Type, Duration (mins), Rate,
# Suppressed Delivery Type, Suppressed Type, Suppressed Rate
basal_cols = [
    "Local Time",
    "Tidepool Data Type",
    "Duration (mins)",
    "Rate",
    "Suppressed Delivery Type",
    "Suppressed Type",
    "Suppressed Rate"
]
basal_df = basal_df[basal_cols].copy()

# For bolus, keep these columns:
# Local Time, Tidepool Data Type, Normal (the bolus amount)
bolus_cols = [
    "Local Time",
    "Tidepool Data Type",
    "Normal"
]
bolus_df = bolus_df[bolus_cols].copy()

# Convert Local Time to datetime for both DataFrames
basal_df["Local Time"] = pd.to_datetime(basal_df["Local Time"], errors="coerce")
bolus_df["Local Time"] = pd.to_datetime(bolus_df["Local Time"], errors="coerce")

# Convert numeric columns appropriately
basal_df["Duration (mins)"] = pd.to_numeric(basal_df["Duration (mins)"], errors="coerce")
basal_df["Rate"] = pd.to_numeric(basal_df["Rate"], errors="coerce")
basal_df["Suppressed Rate"] = pd.to_numeric(basal_df["Suppressed Rate"], errors="coerce")
bolus_df["Normal"] = pd.to_numeric(bolus_df["Normal"], errors="coerce")

# Display initial rows for sanity check
print("Basal Data (first 3 rows):")
display(basal_df.head(3))
print("\nBolus Data (first 3 rows):")
display(bolus_df.head(3))

###############################
# 2) Chunk Basal Data with Flexible Duration (<=6 minutes)
###############################

def chunk_basal_flexible(df_basal, max_chunk_duration=6):
    """
    Splits each basal record into evenly sized chunks such that each chunk's duration 
    is less than or equal to max_chunk_duration minutes.
    
    For each basal record:
      - Let D = Duration (mins)
      - Determine number of chunks n = ceil(D / max_chunk_duration)
      - Each chunk will have duration = D / n minutes (which is <= max_chunk_duration)
      - The delivered insulin "Amount" in each chunk is:
          (Rate in U/hr / 60) * (chunk duration)
        and rounded to 3 decimals.
      - Also, the original Rate is carried over as "Temp rate" for validation.
      
    Returns a DataFrame with the following columns:
      - Local Time: start time of the chunk (increments by chunk duration)
      - Tidepool Data Type: set to "basal"
      - Duration (mins): duration of that chunk (may be fractional)
      - Amount: insulin units delivered in that chunk (rounded to 3 decimals)
      - Temp rate: the original Rate from the basal record
      - Suppressed Delivery Type, Suppressed Type, Suppressed Rate (carried over)
    """
    chunked_rows = []

    for idx, row in df_basal.iterrows():
        start_time = row["Local Time"]
        total_duration = row["Duration (mins)"]
        hourly_rate = row["Rate"]  # in U/hr
        
        # Additional columns to carry over
        suppressed_delivery_type = row["Suppressed Delivery Type"]
        suppressed_type = row["Suppressed Type"]
        suppressed_rate = row["Suppressed Rate"]

        # Skip record if duration is invalid
        if pd.isna(total_duration) or total_duration <= 0:
            continue

        # Determine number of chunks (n) such that each chunk's duration <= max_chunk_duration
        n_chunks = ceil(total_duration / max_chunk_duration)
        # Calculate the actual chunk duration (in minutes)
        chunk_duration = total_duration / n_chunks

        # Calculate rate in U per minute (if rate is available)
        rate_per_minute = np.nan
        if not pd.isna(hourly_rate):
            rate_per_minute = hourly_rate / 60.0

        # Create each chunk row
        for i in range(n_chunks):
            # Calculate start time for this chunk
            chunk_start = start_time + timedelta(minutes=i * chunk_duration)
            # Calculate the amount delivered in this chunk
            chunk_amount = np.nan
            if not pd.isna(rate_per_minute):
                chunk_amount = round(rate_per_minute * chunk_duration, 3)

            new_row = {
                "Local Time": chunk_start,
                "Tidepool Data Type": "basal",
                "Duration (mins)": round(chunk_duration, 3),
                "Amount": chunk_amount,
                "Temp rate": hourly_rate,
                "Suppressed Delivery Type": suppressed_delivery_type,
                "Suppressed Type": suppressed_type,
                "Suppressed Rate": suppressed_rate
            }
            chunked_rows.append(new_row)

    chunked_df = pd.DataFrame(chunked_rows)
    chunked_df.sort_values(by="Local Time", inplace=True)
    chunked_df.reset_index(drop=True, inplace=True)
    return chunked_df

# Apply the flexible basal chunking function (max_chunk_duration set to 6 minutes)
chunked_basal_df = chunk_basal_flexible(basal_df, max_chunk_duration=6)

print("\nChunked Basal Data (first 5 rows):")
display(chunked_basal_df.head(5))
print("Number of rows in chunked basal:", len(chunked_basal_df))

###########################################
# 3) Merge Chunked Basal Data with Bolus Data
###########################################

# Final unified columns:
# [Local Time, Tidepool Data Type, Duration (mins), Amount, Temp rate,
#  Suppressed Delivery Type, Suppressed Type, Suppressed Rate]

final_cols = [
    "Local Time",
    "Tidepool Data Type",
    "Duration (mins)",
    "Amount",
    "Temp rate",
    "Suppressed Delivery Type",
    "Suppressed Type",
    "Suppressed Rate"
]

# For bolus data, rename "Normal" to "Amount" and leave the other fields blank.
bolus_renamed = bolus_df.rename(columns={"Normal": "Amount"}).copy()
bolus_renamed["Duration (mins)"] = ""
bolus_renamed["Temp rate"] = ""
bolus_renamed["Suppressed Delivery Type"] = ""
bolus_renamed["Suppressed Type"] = ""
bolus_renamed["Suppressed Rate"] = ""

# Ensure both DataFrames have the same columns in the same order
bolus_renamed = bolus_renamed[final_cols]
chunked_basal_df = chunked_basal_df[final_cols]

# Concatenate and sort by Local Time
merged_df = pd.concat([chunked_basal_df, bolus_renamed], ignore_index=True)
merged_df.sort_values(by="Local Time", inplace=True)
merged_df.reset_index(drop=True, inplace=True)

print("\nMerged Basal & Bolus Data (first 10 rows):")
display(merged_df.head(10))
print("Total rows in merged DataFrame:", len(merged_df))

##################################
# EXPORT THE MERGED DATA TO CSV
##################################
# Use na_rep='' so that blank cells appear as empty strings.
output_file = "insulin_5min_merged-input-netIOB.csv"
merged_df.to_csv(output_file, index=False, na_rep='')

print(f"\nDone! Exported final merged data to '{output_file}'.")

In [ ]:
# CREATING BASAL PROFILES FOR EACH DAY
"""
Methods – Generation of Daily Basal Insulin Profiles

The merged insulin dataset was first filtered to extract basal insulin records, defined by the "Tidepool Data Type" field.
From these basal records, the date and hour components were extracted from the "Local Time" column. For each day, records
were grouped by hour (0–23) and the average "Suppressed Rate" was computed for each hour after excluding missing values.
To ensure a complete hourly profile, missing hourly values were imputed using forward‐ and backward‐filling.
The resulting hourly averages were then organized into a structured JSON object; each element of the basal profile array 
includes an index, a start time (formatted as "HH:00:00"), the cumulative minute offset, and the computed average insulin 
delivery rate (in IU/hr). Daily profiles were saved as separate JSON files, providing reproducible and scalable inputs 
for subsequent netIOB analyses.
"""

In [ ]:
import pandas as pd
import json
import os

# -------------------------------------------
# CONFIGURATION
# -------------------------------------------
merged_file = "insulin_5min_merged-input-netIOB.csv"
profiles_folder = "daily_profiles"
os.makedirs(profiles_folder, exist_ok=True)

# -------------------------------------------
# READ DATA & FILTER BASAL RECORDS
# -------------------------------------------
insulin_df = pd.read_csv(merged_file, parse_dates=["Local Time"])
# Filter only basal rows (case-insensitive check)
basal_df = insulin_df[insulin_df["Tidepool Data Type"].str.lower() == "basal"].copy()

# -------------------------------------------
# ADD Date and Hour COLUMNS
# -------------------------------------------
basal_df["Date"] = basal_df["Local Time"].dt.date
basal_df["Hour"] = basal_df["Local Time"].dt.hour

# Dictionary to store the filled profiles for each day (for later inspection if needed)
daily_profiles = {}

# -------------------------------------------
# PROCESS EACH DAY: Compute hourly averages (ignoring NaNs) for each day in the dataset
# -------------------------------------------
for day, group in basal_df.groupby("Date"):
    print(f"\nProcessing day: {day}")
    
    # Compute the average "Suppressed Rate" per hour, ignoring NaN values.
    hourly_profile = group.groupby("Hour")["Suppressed Rate"]\
                          .apply(lambda x: x.dropna().mean())\
                          .to_dict()
    
    # Build a template for all 24 hours (0 to 23). Missing hours will be None.
    profile_dict = {hr: hourly_profile.get(hr, None) for hr in range(24)}
    
    # Debug: Print out all suppressed rate values and the computed average for each hour.
    for hr in range(24):
        # Get all raw suppressed rate values for this hour
        values = group[group["Hour"] == hr]["Suppressed Rate"].tolist()
        valid_values = [v for v in values if pd.notna(v)]
        avg_val = profile_dict[hr]
        print(f"  Hour {hr:02d}: values = {valid_values}, average = {avg_val if pd.notna(avg_val) else 'None'}")
    
    # Convert the dictionary to a list (order from hour 0 to 23)
    rates_list = [profile_dict[h] for h in range(24)]
    df_rates = pd.DataFrame({"rate": rates_list})
    # Forward-fill then back-fill missing values
    df_rates["rate"] = df_rates["rate"].ffill().bfill()
    filled_profile = {h: df_rates.loc[h, "rate"] for h in range(24)}
    
    # Build the "basalprofile" array in the structure expected by netIOB.
    basal_rate_structure_list = []
    for hr in range(24):
        rate_val = round(float(filled_profile[hr]), 3)
        entry = {
            "i": hr,
            "start": f"{hr:02d}:00:00",
            "minutes": hr * 60,
            "rate": rate_val
        }
        basal_rate_structure_list.append(entry)
    
    # Construct the final profile dictionary.
    profile_data = {
        "dia": 5,
        "basalprofile": basal_rate_structure_list
    }
    
    # Save this daily profile as a JSON file.
    profile_filename = os.path.join(profiles_folder, f"profile_{day}.json")
    with open(profile_filename, "w") as f:
        json.dump(profile_data, f, indent=2)
    
    daily_profiles[str(day)] = filled_profile
    print(f"Daily basal profile for {day} saved to {profile_filename}")

print("\nDaily basal profiles created for:")
print(list(daily_profiles.keys()))

In [ ]:
# oref0 checks

import subprocess
import os

# Check if Node.js is installed by calling "node -v"
try:
    node_version = subprocess.check_output(["node", "-v"]).decode("utf-8").strip()
    print("Node.js is installed, version:", node_version)
except Exception as e:
    print("Node.js does not appear to be installed. Please install Node.js to continue.")
    # Optionally, you can raise an error or exit:
    # raise SystemExit("Node.js is required for netIOB computation.")

# Define the expected path to the oref0-calculate-iob.js script
oref0_script_path = os.path.join("oref0", "bin", "oref0-calculate-iob.js")

# Check if the oref0 script exists at the expected location
if os.path.exists(oref0_script_path):
    print("Found oref0 script at:", oref0_script_path)
else:
    print(f"Error: oref0 script not found at expected location: {oref0_script_path}")
    # Provide a message or instructions on where to get it:
    print("Please ensure you have cloned the oref0 repository from https://github.com/openaps/oref0")

In [ ]:
# Clone the oref0 repository into a folder named 'oref0'
!git clone https://github.com/openaps/oref0.git

In [ ]:
# More oref0

import os

oref0_script_path = os.path.join("oref0", "bin", "oref0-calculate-iob.js")
if os.path.exists(oref0_script_path):
    print("oref0 script successfully cloned at:", oref0_script_path)
else:
    print("Error: oref0 script still not found at expected location:", oref0_script_path)

In [ ]:
#install moment-timezone

!cd "oref0" && npm install moment-timezone

In [ ]:
# netIOB
"""
Methods – Computation of Net Insulin On Board (netIOB) Using Daily Basal Profiles

The merged insulin dataset is first ingested from a CSV file containing 5‐minute insulin data. 
The dataset is sorted by its local timestamp and an additional ‘date_str’ column is derived by 
extracting the date component from the "Local Time" field. For each day, the data are processed 
individually. A day-specific basal profile JSON file, previously generated, is used as an input 
to the netIOB computation.

For each day, the method iterates over all 5‐minute intervals and, for each interval, extracts the 
events that occurred in the prior 24 hours (using a helper function that filters the data based on 
timestamp). The filtered events are then formatted into a “pumphistory” JSON structure that matches 
the expected input format of the external netIOB computation script. The system creates a minimal 
clock file that contains the current 5‐minute interval’s timestamp (with timezone offset) and calls 
the Node.js script (oref0-calculate-iob.js) with the pump history, the corresponding day‐specific 
basal profile, and the clock file as parameters. The computed netIOB value is then stored in a 
dictionary indexed by the row number.

After processing all rows for each day, the computed netIOB values are merged back into the original 
DataFrame, which is then exported as a CSV file containing the complete time series with an added 
"netIOB" column. This approach allows for day‐by‐day processing and enables partial output recovery 
if the computation is interrupted.
"""

In [ ]:
import pandas as pd
import numpy as np
import json
import subprocess
import os
from datetime import datetime, timedelta
from math import ceil
import multiprocessing

# Force the start method to 'fork' (only works on Unix-based systems)
multiprocessing.set_start_method("fork", force=True)

# ---------------------------------------
# CONFIGURATION
# ---------------------------------------
merged_file = "insulin_5min_merged-input-netIOB.csv"  # The 5-min insulin data
profiles_folder = "daily_profiles"                    # Folder with day-specific profile JSON files
oref0_script_path = "oref0/bin/oref0-calculate-iob.js"  # Path to the local oref0 script
final_output_file = "insulin_5min_merged-output-with-netIOB.csv"

# Can run different date series, eg test with 1-2 days first to make sure it works before running on the full series
start_date = pd.to_datetime("2022-01-02").date()
end_date   = pd.to_datetime("2022-02-15").date()

# Create folders for partial outputs and temporary JSON files.
partial_folder = "partial_outputs"
os.makedirs(partial_folder, exist_ok=True)
temp_dir = "temp"
os.makedirs(temp_dir, exist_ok=True)

# ---------------------------------------
# HELPER FUNCTION: Build a Day-Long Pump History
# ---------------------------------------
def build_day_history(df_all, day_str):
    """
    Build a pumpHistory list for a given day.
    The history includes all events from (day_start - 24h) to (day_end).
    Returns a list of dictionaries in the oref0 pumpHistory format.
    IMPORTANT: The events are sorted in descending order (most recent first)
    as expected by the oref0 netIOB calculation.
    """
    day_start = pd.Timestamp(day_str)
    day_end = day_start + timedelta(days=1)
    # Filter for events in the lookback window up to the end of the day.
    history_df = df_all[(df_all["Local Time"] >= (day_start - timedelta(hours=24))) & 
                        (df_all["Local Time"] < day_end)].copy()
    # Sort in descending order (most recent first)
    history_df = history_df.sort_values("Local Time", ascending=False)
    
    pumphistory_data = []
    for _, row in history_df.iterrows():
        hist_time_str = row["Local Time"].strftime("%Y-%m-%dT%H:%M:%S+00:00")
        ttype = str(row["Tidepool Data Type"]).lower()
        if ttype == "basal":
            duration_mins = float(row["Duration (mins)"]) if not pd.isna(row["Duration (mins)"]) else 0.0
            pumphistory_data.append({
                "timestamp": hist_time_str,
                "_type": "TempBasalDuration",
                "duration (min)": duration_mins
            })
            basal_rate = float(row["Temp rate"]) if not pd.isna(row["Temp rate"]) else 0.0
            pumphistory_data.append({
                "timestamp": hist_time_str,
                "_type": "TempBasal",
                "temp": "absolute",
                "rate": round(basal_rate, 3)
            })
        elif ttype == "bolus":
            bolus_amt = float(row["Amount"]) if not pd.isna(row["Amount"]) else 0.0
            pumphistory_data.append({
                "timestamp": hist_time_str,
                "_type": "Bolus",
                "amount": bolus_amt,
                "programmed": bolus_amt,
                "unabsorbed": 0,
                "duration": 0
            })
    return pumphistory_data

# ---------------------------------------
# HELPER FUNCTION: Compute netIOB for a Single Timestamp (Worker Function)
# ---------------------------------------
def compute_iob_for_timestamp(current_time, history_file, day_profile_file, temp_dir, oref0_script_path):
    """
    Writes a unique clock file for the current_time, calls the oref0 Node.js script,
    and returns the computed netIOB value.
    """
    clock_str = current_time.strftime("%Y-%m-%dT%H:%M:%S+00:00")
    unique_clock_file = os.path.join(temp_dir, f"clock_{current_time.timestamp()}_{os.getpid()}.json")
    with open(unique_clock_file, "w") as cf:
        json.dump(clock_str, cf)
    try:
        output = subprocess.check_output([
            "node",
            oref0_script_path,
            history_file,        # full pump history file
            day_profile_file,      # day-specific profile
            unique_clock_file
        ])
        output_json = json.loads(output)
        iob_val = output_json[0]["iob"]
    except Exception as e:
        print(f"Error computing netIOB for time {current_time}: {e}")
        iob_val = 0.0
    os.remove(unique_clock_file)
    return iob_val

# ---------------------------------------
# HELPER FUNCTION: Process netIOB for a Single Day Using Multiprocessing
# ---------------------------------------
def compute_netiob_for_day(day_data, history_file, day_profile_file, temp_dir, oref0_script_path):
    times = day_data["Local Time"].tolist()
    args = [(t, history_file, day_profile_file, temp_dir, oref0_script_path) for t in times]
    with multiprocessing.Pool() as pool:
        results = pool.starmap(compute_iob_for_timestamp, args)
    return results

# ---------------------------------------
# MAIN FUNCTION: PROCESS netIOB Iteratively by Day (Test Range)
# ---------------------------------------
def compute_netiob_daily_profiles_iterative():
    df = pd.read_csv(merged_file, parse_dates=["Local Time"])
    df.sort_values("Local Time", inplace=True)
    df.reset_index(drop=True, inplace=True)
    df["date_str"] = df["Local Time"].dt.date.astype(str)
    
    all_dates = sorted(df["date_str"].unique())
    filtered_dates = [d for d in all_dates if pd.to_datetime(d).date() >= start_date 
                      and pd.to_datetime(d).date() <= end_date]
    
    output_dfs = []
    processed_days = []
    
    for day_str in filtered_dates:
        start_day_time = datetime.now()
        print(f"\nProcessing date {day_str}... (start: {start_day_time.strftime('%Y-%m-%d %H:%M:%S')})")
        
        day_data = df[df["date_str"] == day_str].copy()
        # Build pump history (lookback from day_start - 24h to day_end)
        history = build_day_history(df, day_str)
        history_file = os.path.join(temp_dir, f"pumphistory_{day_str}.json")
        with open(history_file, "w") as phf:
            json.dump(history, phf, indent=2)
        print(f"  Pump history for {day_str} written to {history_file} with {len(history)} events.")
        
        day_profile_file = os.path.join(profiles_folder, f"profile_{day_str}.json")
        if not os.path.exists(day_profile_file):
            print(f"Warning: No profile file for {day_str}. Setting netIOB=0 for all rows.")
            netiob_list = [0.0] * len(day_data)
        else:
            netiob_list = compute_netiob_for_day(day_data, history_file, day_profile_file, temp_dir, oref0_script_path)
        
        day_data["netIOB"] = netiob_list
        partial_file = os.path.join(partial_folder, f"partial_{day_str}_with_netiob.csv")
        day_data.to_csv(partial_file, index=False)
        print(f"  Saved partial output for {day_str} to {partial_file}")
        
        end_day_time = datetime.now()
        duration = end_day_time - start_day_time
        print(f"Finished processing {day_str} at {end_day_time.strftime('%Y-%m-%d %H:%M:%S')}. Duration: {duration}")
        
        output_dfs.append(day_data)
        processed_days.append(day_str)
    
    final_df = pd.concat(output_dfs, ignore_index=True)
    final_df.to_csv(final_output_file, index=False)
    print(f"\nAll done. Final netIOB data saved to {final_output_file}.")
    print("Processed days:", processed_days)

if __name__ == '__main__':
    compute_netiob_daily_profiles_iterative()